In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd /content/drive/MyDrive/assessmentanalysis/

/content/drive/MyDrive/assessmentanalysis


In [ ]:
#Install the required packages for this project
!pip install --upgrade pip
!pip install --upgrade transformers datasets[audio] accelerate
!pip install ffmpeg-python
!pip install firebase-admin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 57.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 61.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


## Data Extration

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline , AutoProcessor, AutoModelForCausalLM
import cv2
import os
from PIL import Image

In [ ]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import storage
from firebase_admin import firestore

cred = credentials.Certificate('videoanalysis-d5eb4-firebase-adminsdk-u5vqo-754ba234c2.json')
firebase_admin.initialize_app(cred, {
    'storageBucket': 'videoanalysis-d5eb4.firebasestorage.app'
})

bucket = storage.bucket()
db = firestore.client()

In [ ]:
file_name="Nextjs14Tutorial10NotFoundPage.mp4"

In [ ]:
if os.path.exists(file_name):
  print(f"The file '{file_name}' exists.")
else:
  video_path_in_firebase = "videos/"+file_name  # Path
  local_video_path = file_name  # Local path to download
  blob = bucket.blob(video_path_in_firebase)  # Get blob
  blob.download_to_filename(local_video_path)
  print(f"Video downloaded to: {local_video_path}")

The file 'Nextjs14Tutorial10NotFoundPage.mp4' exists.


In [ ]:
video_name = os.path.basename(file_name)
video_name_without_extension = os.path.splitext(video_name)[0]

In [ ]:
if not os.path.exists(video_name_without_extension):

  def extract_frames_from_video(video_path, output_folder, interval_seconds=15):
    """Extracts frames from a video at a specified interval.

    Args:
      video_path: Path to the input video file.
      output_folder: Path to the folder where extracted frames will be saved.
      interval_seconds: The time interval (in seconds) between extracted frames.
    """

    if not os.path.exists(output_folder):
      os.makedirs(output_folder)

    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(fps * interval_seconds)

    while True:
      ret, frame = cap.read()
      if not ret:
        break

      if frame_count % frame_interval == 0:
        output_path = os.path.join(output_folder, f"frame_{frame_count}.png")
        cv2.imwrite(output_path, frame)
        print(f"Frame saved: {output_path}")

      frame_count += 1

    cap.release()

  # Using extract_frames_from_video function on the video
  extract_frames_from_video(file_name, video_name_without_extension)

Frame saved: Nextjs14Tutorial10NotFoundPage/frame_0.png
Frame saved: Nextjs14Tutorial10NotFoundPage/frame_450.png
Frame saved: Nextjs14Tutorial10NotFoundPage/frame_900.png
Frame saved: Nextjs14Tutorial10NotFoundPage/frame_1350.png
Frame saved: Nextjs14Tutorial10NotFoundPage/frame_1800.png
Frame saved: Nextjs14Tutorial10NotFoundPage/frame_2250.png
Frame saved: Nextjs14Tutorial10NotFoundPage/frame_2700.png
Frame saved: Nextjs14Tutorial10NotFoundPage/frame_3150.png
Frame saved: Nextjs14Tutorial10NotFoundPage/frame_3600.png
Frame saved: Nextjs14Tutorial10NotFoundPage/frame_4050.png
Frame saved: Nextjs14Tutorial10NotFoundPage/frame_4500.png
Frame saved: Nextjs14Tutorial10NotFoundPage/frame_4950.png
Frame saved: Nextjs14Tutorial10NotFoundPage/frame_5400.png
Frame saved: Nextjs14Tutorial10NotFoundPage/frame_5850.png
Frame saved: Nextjs14Tutorial10NotFoundPage/frame_6300.png
Frame saved: Nextjs14Tutorial10NotFoundPage/frame_6750.png
Frame saved: Nextjs14Tutorial10NotFoundPage/frame_7200.png
Fr

In [ ]:
# Check and insert in the 'caption' collection
caption_doc_ref = db.collection('caption').document(video_name_without_extension)
caption_doc = caption_doc_ref.get()

In [ ]:
model_id = 'microsoft/Florence-2-large'
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True).eval().cuda()
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.44k [00:00<?, ?B/s]

configuration_florence2.py:   0%|          | 0.00/15.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large:
- configuration_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_florence2.py:   0%|          | 0.00/127k [00:00<?, ?B/s]

...

processing_florence2.py:   0%|          | 0.00/48.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large:
- processing_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_config.json:   0%|          | 0.00/34.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models, datasets
import os
from PIL import Image
from sklearn.model_selection import train_test_split
import shutil
import numpy as np
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data_transforms = {
   'val': transforms.Compose([
       transforms.Resize(256),
       transforms.CenterCrop(224),
       transforms.ToTensor(),
       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
   ])
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model structure
loaded_model = models.resnet18(pretrained=False)
num_ftrs = loaded_model.fc.in_features
loaded_model.fc = nn.Linear(num_ftrs, 2)

# Load the trained weights
loaded_model.load_state_dict(torch.load('code_detect.pth'))
loaded_model = loaded_model.to(device)
loaded_model.eval()  # Set to evaluation mode

def predict_image(image_path, model):
    """
    Predicts the class of an image using the provided model.

    Args:
    image_path (str): Path to the image file.
    model (torch.nn.Module): The trained model.

    Returns:
    str: The predicted class name.
    """
    image_temp = Image.open(image_path).convert('RGB')
    image_temp = data_transforms['val'](image_temp).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image_temp)
        _, preds = torch.max(outputs, 1)

    class_names = ['Code', 'Other']  # Define class names for classification
    return class_names[preds[0]]

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
<ipython-input-14-03f6663b5bb0>:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the function

In [ ]:
def transcribe_video_OCR(image):
    inputs = processor(text='<OCR>', images=image, return_tensors="pt")
    generated_ids = model.generate(
      input_ids=inputs["input_ids"].cuda(),
      pixel_values=inputs["pixel_values"].cuda(),
      max_new_tokens=256,
      early_stopping=False,
      do_sample=False,
      num_beams=3,
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(
        generated_text,
        task='<OCR>',
        image_size=(image.width, image.height)
    )

    return parsed_answer['<OCR>']

In [ ]:
def transcribe_video_caption(image):
    inputs = processor(text='<DETAILED_CAPTION>', images=image, return_tensors="pt")
    generated_ids = model.generate(
      input_ids=inputs["input_ids"].cuda(),
      pixel_values=inputs["pixel_values"].cuda(),
      max_new_tokens=256,
      early_stopping=False,
      do_sample=False,
      num_beams=3,
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(
        generated_text,
        task='<DETAILED_CAPTION>',
        image_size=(image.width, image.height)
    )

    return parsed_answer['<DETAILED_CAPTION>']

In [ ]:
all_caps=[]
for filename in os.listdir(video_name_without_extension):
  image_path = os.path.join(video_name_without_extension, filename)
  img_type=predict_image(image_path, loaded_model)
  img=Image.open(image_path)
  if img_type=='Code':
    print("Code Detected")
    text=transcribe_video_OCR(img)
    caption_data = {
          "name":filename,
          "text": text,
          "type": 'Code'
    }
    all_caps.append(caption_data)
  else:
    print("Other image Detected")
    text=transcribe_video_caption(img)
    caption_data = {
          "name":filename,
          "text": text,
          "type": 'Other'
    }
    all_caps.append(caption_data)

caption_doc_ref.set({"filename": video_name_without_extension,"data":all_caps})

Other image Detected
Other image Detected
Code Detected
Code Detected
Code Detected
Code Detected
Code Detected
Code Detected
Other image Detected
Other image Detected
Code Detected
Code Detected
Code Detected
Code Detected
Code Detected
Other image Detected
Code Detected
Code Detected


update_time {
  seconds: 1733150907
  nanos: 165124000
}